In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, Border, Side

# Input/output
input_file = 'results_with_choose_distance_and_lane.xlsx'
output_file = 'Ketaki_PoconoTest.xlsx'

# Read the data
df = pd.read_excel(input_file)

# Table anchors: (row, col) in Excel numbering for top-left of each table
anchors = [
    (4, 3),    # C4
    (4, 20),   # T4
    (23, 3),   # C23
    (23, 20),  # T23
]

# Create workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Line Up Sheet"

# Title setup (Race and Date)
ws.merge_cells(start_row=2, start_column=3, end_row=2, end_column=7)
cell = ws.cell(row=2, column=3, value="Race: 25POC")
cell.font = Font(bold=True, size=16)
cell.alignment = Alignment(horizontal='left', vertical='center')

ws.merge_cells(start_row=2, start_column=14, end_row=2, end_column=15)
date_label = ws.cell(row=2, column=14, value="Date")
date_label.alignment = Alignment(horizontal='center', vertical='center')
date_label.font = Font(bold=True, size=14)

ws.merge_cells(start_row=2, start_column=19, end_row=2, end_column=21)
date_value = ws.cell(row=2, column=19, value="6/22/25")
date_value.alignment = Alignment(horizontal='center', vertical='center')
date_value.font = Font(bold=True, size=14)

# Borders and fonts
thick = Side(border_style="thick", color="000000")
thin = Side(border_style="thin", color="000000")
header_font = Font(bold=True, size=12)
lap_font = Font(bold=True, size=11)
center = Alignment(horizontal='center', vertical='center')

# Get first 4 laps (anchors are set for 4 tables)
lap_numbers = sorted(df['Lap'].unique())[:4]

for i, lap in enumerate(lap_numbers):
    anchor_row, anchor_col = anchors[i]
    # Lap label
    ws.merge_cells(start_row=anchor_row, start_column=anchor_col, end_row=anchor_row, end_column=anchor_col+3)
    lap_label = ws.cell(row=anchor_row, column=anchor_col, value=f"Lap {lap}")
    lap_label.font = lap_font
    lap_label.alignment = center

    # Header
    ws.cell(row=anchor_row+1, column=anchor_col, value="Driver")
    ws.cell(row=anchor_row+1, column=anchor_col+1, value="Inside")
    ws.cell(row=anchor_row+1, column=anchor_col+2, value="Outside")
    for h in range(3):
        head = ws.cell(row=anchor_row+1, column=anchor_col+h)
        head.font = header_font
        head.alignment = center

    # 16 rows for each position
    lap_df = df[df["Lap"] == lap].sort_values(by="Position_Before_Choose")
    for pos in range(1, 17):
        ws.cell(row=anchor_row+1+pos, column=anchor_col-1, value=pos)
        ws.cell(row=anchor_row+1+pos, column=anchor_col-1).alignment = center

        driver_info = ""
        inside = outside = ""
        driver_row = lap_df[lap_df["Position_Before_Choose"] == pos]
        if not driver_row.empty:
            driver_info = str(driver_row.iloc[0]["Driver"])  # You can use more fields if needed
            lane = driver_row.iloc[0]["Lane Choice"].strip().lower()
            if lane == "inside":
                inside = driver_info
            elif lane == "outside":
                outside = driver_info
        ws.cell(row=anchor_row+1+pos, column=anchor_col, value=driver_info)  # driver info in Driver column
        ws.cell(row=anchor_row+1+pos, column=anchor_col+1, value=inside)
        ws.cell(row=anchor_row+1+pos, column=anchor_col+2, value=outside)
        # Center all
        ws.cell(row=anchor_row+1+pos, column=anchor_col).alignment = center
        ws.cell(row=anchor_row+1+pos, column=anchor_col+1).alignment = center
        ws.cell(row=anchor_row+1+pos, column=anchor_col+2).alignment = center

    # Draw thick border for outer box, thin for inner
    for r in range(anchor_row, anchor_row+18):
        for c in range(anchor_col-1, anchor_col+3):
            cell = ws.cell(row=r, column=c)
            left = thick if c == anchor_col-1 else thin
            right = thick if c == anchor_col+2 else thin
            top = thick if r == anchor_row else thin
            bottom = thick if r == anchor_row+17 else thin
            cell.border = Border(left=left, right=right, top=top, bottom=bottom)
            cell.alignment = center  # Center all cells

# Set column widths for tidy appearance
for col in range(3, 24):
    ws.column_dimensions[chr(64+col)].width = 4.5
ws.column_dimensions['B'].width = 5
ws.column_dimensions['S'].width = 5

wb.save(output_file)
print(f"Created {output_file} in requested format.")

Created Ketaki_PoconoTest.xlsx in requested format.
